In [ ]:
# attention의 구조

In [1]:
import tensorflow as tf
from tensorflow.keras import Model, layers

In [4]:
# 인코더 - eng
enc_input = layers.Input(shape = (4,171))
enc_output, state_h, state_c =layers.LSTM(128, return_sequences=True, return_state=True)(enc_input)

# 디코더 - kor
dec_input = layers.Input(shape = (3,171)) # 1-start토큰 + 문자2
dec_output, _, _ =layers.LSTM(128, return_sequences=True, return_state=True)(dec_input, initial_state=[state_h, state_c])


In [6]:
# 어텐션 매커니즘
context_vector = layers.Attention()([dec_output, enc_output])
context_and_lstm_output = layers.Concatenate()([context_vector, dec_output])

In [7]:
# 디코더의 출력층 정의
output = layers.Dense(171,activation='softmax')(context_and_lstm_output)

In [10]:
model = Model(inputs=[enc_input, dec_input], outputs=[output])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 4, 171)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 3, 171)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 4, 128),  │    153,600 │ input_layer_3[0]… │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 3, 128),  │    153,600 │ input_layer_4[0]… │
│                     │ (None, 128),      │            │ lstm_2[0][1],     │
│                     │ (None, 128)]      │            │ lstm_2[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_1         │ (None, 3, 128)    │          0 │ lstm_3[0][0],     │
│ (Attention)         │                   │            │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 3, 256)    │          0 │ attention_1[0][0… │
│ (Concatenate)       │                   │            │ lstm_3[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 3, 171)    │     43,947 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 351,147 (1.34 MB)

 Trainable params: 351,147 (1.34 MB)

 Non-trainable params: 0 (0.00 B)

# eng -> kor

In [11]:
# 손실함수와 옵디마이저 지정
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')

In [13]:
import pandas as pd
import numpy as np

arr1 = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz'] # 전체문자배열
arr2 = pd.read_csv('korean.csv',header=None)

num_to_char = arr1 + arr2[0].values.tolist()
len(num_to_char)

171

In [ ]:
# 학습용 데이터셋 설정

In [ ]:
# 변환행렬 - 딕셔너리
char_to_num = {char:i for i, char in enumerate(num_to_char)}
char_to_num

{'S': 0,
 'E': 1,
 'P': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28,
 '가': 29,
 '각': 30,
 '간': 31,
 '감': 32,
 '개': 33,
 '거': 34,
 '것': 35,
 '게': 36,
 '계': 37,
 '고': 38,
 '관': 39,
 '광': 40,
 '구': 41,
 '굴': 42,
 '규': 43,
 '그': 44,
 '금': 45,
 '기': 46,
 '깊': 47,
 '나': 48,
 '날': 49,
 '남': 50,
 '내': 51,
 '넓': 52,
 '녀': 53,
 '노': 54,
 '놀': 55,
 '농': 56,
 '높': 57,
 '뉴': 58,
 '늦': 59,
 '다': 60,
 '단': 61,
 '도': 62,
 '동': 63,
 '들': 64,
 '람': 65,
 '랑': 66,
 '래': 67,
 '램': 68,
 '류': 69,
 '름': 70,
 '릎': 71,
 '리': 72,
 '많': 73,
 '망': 74,
 '매': 75,
 '머': 76,
 '먼': 77,
 '멍': 78,
 '메': 79,
 '명': 80,
 '모': 81,
 '목': 82,
 '무': 83,
 '물': 84,
 '미': 85,
 '바': 86,
 '반': 87,
 '방': 88,
 '번': 89,
 '복': 90,
 '부': 91,
 '분': 92,
 '붕': 93,
 '비': 94,
 '뿌': 95,
 '사': 96,
 '상': 97,
 '색': 98,
 '생': 99,
 '서': 100,

In [17]:
raw = pd.read_csv('translate.csv',header=None)
eng_kor = raw.values.tolist()
print(len(eng_kor))

110


In [ ]:
eng_kor[0]

['cold', '감기']

# 단어 -> 숫자로 변환

In [25]:
temp_eng = 'love'
temp_eng_n = [char_to_num[c] for c in temp_eng] # [14, 17, 24, 7]
np.eye(171)[temp_eng_n] #shape 맞추기

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.,

In [26]:
# 단어를 원-핫 인코딩된 배열로 변환하는 함수
def encode(eng_kor):#번역데이터
    enc_in = []   # 인코더 입력 (영어 단어 → 원-핫 벡터)
    dec_in = []   # 디코더 입력 (한국어 단어 앞에 시작 토큰 'S' 추가)
    rnn_out = []  # 디코더 출력 (한국어 단어 뒤에 종료 토큰 'E' 추가)

    for seq in eng_kor:
        # 1. 영어 단어를 숫자 인덱스로 변환 후 원-핫 인코딩
        eng = [char_to_num[c] for c in seq[0]]
        enc_in.append(np.eye(171)[eng])   # 171차원 one-hot 벡터

        # 2. 한국어 단어 앞에 'S' 추가 후 인덱스로 변환 → 디코더 입력
        kor = [char_to_num[c] for c in ('S' + seq[1])]
        dec_in.append(np.eye(171)[kor])   # 디코더 입력도 원-핫 인코딩

        # 3. 한국어 단어 뒤에 'E' 추가 후 인덱스로 변환 → 디코더 출력(정답)
        target = [char_to_num[c] for c in (seq[1] + 'E')]
        rnn_out.append(target)

    # 리스트를 넘파이 배열로 변환
    enc_in = np.array(enc_in)      # (샘플 수, 단어 길이, 171)
    dec_in = np.array(dec_in)      # (샘플 수, 단어 길이+1, 171)
    rnn_out = np.array(rnn_out)    # (샘플 수, 단어 길이+1)

    # 디코더 출력에 차원 추가 → (샘플 수, 단어 길이+1, 1)
    rnn_out = np.expand_dims(rnn_out, axis=2)

    return enc_in, dec_in, rnn_out

In [29]:
sample = [['word', '단어']]
encode(sample)

(array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [30]:
X_enc, X_dec, y_rnn = encode(eng_kor)
model.fit([X_enc, X_dec],y_rnn, epochs=100)


Epoch 1/100


2025-09-05 11:28:39.338291: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91200


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 5.1321
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 5.0913
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.0439
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 4.9740
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 4.8531
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 4.6134
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 4.1082
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 3.5049
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 3.5968
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 3.4551
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 3.3467
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 3.3617
Epoch 13/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 3.3189
Epoch 14/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 3.2680
Epoch 15/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 3.2572
Epoch 16/100
4/4 ━━━━━━━━━━━━━━

# 테스트

In [36]:
enc_in, doc_in, _ =encode([['tall','PP']])

In [ ]:
pred = model.predict([enc_in, doc_in])
word = np.argmax(pred[0], axis=1) # [152,  60,   1]
num_to_char[word[0]],num_to_char[word[1]], num_to_char[word[2]] # ('크', '다', 'E')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


('크', '다', 'E')

In [ ]:
# 여러 단어 [] -> 출력 하는 프로그램 완성

In [ ]:
word = ['plan','life','free','tall']

for:
    encode()
    model.predict()
    word
    # 글자를 합치는 단계
